In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
import haversine as hs
import scipy.stats as st
import seaborn as sns
sns.set()

from sklearn.neighbors import LocalOutlierFactor
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

pio.renderers.default = "notebook"

## Suppress scientific notation
pd.options.display.float_format = '{:.4f}'.format

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('./dataset.csv')
df.head()

,DriverId,EventName,Latitude,Longitude,Speed km/h,ts
0,0,Timed Event,34.1866,-118.0881,64.0000,2017-11-01 00:00:02.430
1,0,Distance Event,34.1861,-118.0892,53.0000,2017-11-01 00:00:05.600
2,0,Distance Event,34.1864,-118.0896,34.0000,2017-11-01 00:00:13.640
3,0,Distance Event,34.1875,-118.0889,33.0000,2017-11-01 00:00:26.070
4,0,Distance Event,34.1887,-118.0865,32.0000,2017-11-01 00:00:35.090


In [4]:
df['distance'] = df.apply(lambda x: hs.haversine((x['Latitude'], x['Longitude']), (x['Latitude'], x['Longitude'])), axis=1)
df.head()

Error: Canceled future for execute_request message before replies were done